In [ ]:
%matplotlib qt
import numpy as np
import hyperspy.api as hs
from ncempy.io import dm
import matplotlib.pyplot as plt
from scipy.signal import peak_widths, find_peaks
import math

In [ ]:
#convert text files
def txtconverter(numpy_array):
    file = str(numpy_array).replace('[','')
    file = file.replace(']','')
    data = np.fromstring(file, sep=',')
    return data

#sorting data into counts and eV
def find_counts(data):
    counts = data[1:-1:2]
    return counts

def find_ev(data):
    ev = data[0:-1:2]
    return ev

#plot the spectrum with HyperSpy
def hyperspy_plot(ev, counts):
    s = hs.signals.EELSSpectrum(counts)
    s.axes_manager[0].scale = np.diff(ev).mean()
    s.axes_manager[0].unit = 'eV'
    s.axes_manager[0].offset = ev[0]
    s.axes_manager[0].name = 'Energy'
    return s

#FWHM comparisons
def FWHM_testing(alpha, gamma, hs_signal, hs_deconvolved, height):
    
    peaks1, _ = find_peaks(hs_signal, height=1)
    results_half_signal = peak_widths(hs_signal, peaks1, rel_height=0.5)
    
    peaks2, _ = find_peaks(hs_deconvolved, height=height)
    results_half_deconvolved = peak_widths(hs_deconvolved, peaks2, rel_height=0.5)
    
    FWHM_signal = 4 / 1000 * results_half_signal[0]
    FWHM_deconvolved = 4 / 1000 * results_half_deconvolved[0]
    
    Lorentzian_FWHM = 2 * gamma
    Gaussian_FWHM = 2 * alpha
    
    relative_error = abs((FWHM_deconvolved[0] - Lorentzian_FWHM)/Lorentzian_FWHM*100)
    
    print("FWHM of signal =", FWHM_signal[0], "eV", 
          "\nFWHM of deconvolved =", FWHM_deconvolved[0], "eV", 
          "\nFWHM of Lorentzian =", Lorentzian_FWHM, "eV", 
          "\nRelative error =",  math.trunc(relative_error), "%\n")
    
#plotting the noise
def noise(Spectrum, deconvolved, PSF):
    noise = np.subtract(Spectrum, np.convolve(deconvolved, PSF, mode='same'))
    return noise

#Richardson-Lucy algorithm (code from Edson Bellido)
def RL(iterations, PSF, Spectrum):
    RL4 = np.copy(Spectrum)
    for i in range(iterations):
        RL1 = np.convolve(PSF, RL4, mode='same')
        RL2 = np.divide(Spectrum,RL1)
        RL3 = np.convolve(PSF, RL2, mode='same')
        RL4 = np.multiply(RL3, RL4)
    return RL4

MEM from the Hollis article 1992 derived without any assumptions about noise and using the Lagrangian multiplier approach. 

In [ ]:
def MEM(iterations, PSF, Spectrum):
    Lambda = [0]*len(PSF)
    
    PSF = np.divide(PSF, np.sum(PSF))
    for i in range(iterations):
        O = np.fft.ifft(np.multiply(np.fft.fft(PSF), np.fft.fft(Lambda)))
        O = np.exp(O)
        O = np.divide(O, np.sum(O))
        S = np.sum(np.multiply(O, np.log(O)))
        I = np.fft.ifft(np.multiply(np.fft.fft(PSF), np.fft.fft(O)))
        scale = np.divide(np.sum(Spectrum), np.sum(I))
        Lambda = np.subtract(np.add(Lambda, np.log(np.divide(Spectrum, scale))), np.log(I))
    return np.multiply(O, scale)

In [ ]:
#load file as numpy array
Signal = np.loadtxt("D:\Downloads\Signal1.txt",dtype="str")
PSF = np.loadtxt("D:\Downloads\PSF1.txt", dtype='str')
Real = np.loadtxt("D:\Downloads\Real1.txt", dtype='str')

#convert text file to usable numpy array
signal = txtconverter(Signal)
psf = txtconverter(PSF)
real = txtconverter(Real)

#separate data into counts and ev
signal_counts = find_counts(signal)
psf_counts = find_counts(psf)
real_counts = find_counts(real)
ev = find_ev(signal)

In [ ]:
RL_deconvolve = RL(10, psf_counts, signal_counts)
s_RL = hyperspy_plot(ev, RL_deconvolve)

MEM_deconvolve = MEM(100, psf_counts, signal_counts)
s_MEM = hyperspy_plot(ev, MEM_deconvolve)

s_signal = hyperspy_plot(ev, signal_counts)

In [ ]:
print("RL")
FWHM_RL = FWHM_testing(0.1, 0.1, s_signal, s_RL, 1)

print("MEM")
#FWHM_MEM = FWHM_testing(0.1, 0.1, s_signal, s_MEM, 1)

In [ ]:
s_signal.plot()

In [ ]:
s_RL.plot()

In [ ]:
s_MEM.plot()